In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time

def scrape_twitter_links(page, xpath):
    page.find_element_by_xpath(xpath).click()
    soup = BeautifulSoup(page.page_source, 'html.parser')
    hyperlinks = [subSoup.find_all('a') for subSoup in soup.find_all('div', {'class':'news-update is-injured'})]
    hyperlinks = [links for nested_hyperlinks in hyperlinks for links in nested_hyperlinks]
    hyperlinks = [subSoup['href'] for subSoup in hyperlinks if 'twitter' in subSoup['href']]
    accounts = set([twitterLink.split('/')[3] for twitterLink in hyperlinks])
    accounts = {accountNames + '\n' for accountNames in accounts}
    return accounts, hyperlinks

page = webdriver.Firefox()
page.get('https://www.rotowire.com/baseball/news.php?injuries=all')

time.sleep(10)

accounts, hyperlinks = scrape_twitter_links(page, '/html/body/div[1]/div/main/div[1]/div/div[2]/div/a[6]')
time.sleep(2)
accounts2, hyperlinks2 = scrape_twitter_links(page, '/html/body/div[1]/div/main/div[1]/div/div[2]/div/a[7]')

accounts = accounts.union(accounts2)
hyperlinks = hyperlinks + hyperlinks2

page.close()

file = open('accountList.txt', 'r')
text = file.readlines()
file.close()

with open('accountList.txt', 'w') as myfile:
    for twitterAccounts in set(text).union(accounts):
        myfile.write(twitterAccounts)
        
print(len(set(text).union(accounts)))
        
with open('labeledTweets.txt', 'w') as myfile:
    for labeledData in set([links + '\n' for links in hyperlinks]):
        myfile.write(labeledData)

2212
